## Importing necessary libs

In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import json

## Observing data format

In [2]:
df = pd.read_csv("data.csv").drop('Unnamed: 0', 1)

In [3]:
df.head()

,userID_anonymized,arrival_time,event_time,eventType,event_data,platform,session_no,wifi_state,model,operating_system,totalrevenue
0,12acc8df-d4f9,11/30/20 18:01,12/26/20 3:16,level_event,"{duration=30.328, level=47, progress=36, play_...",ios,12,0,"iPhone12,8",iOS 14.2,0.870634
1,12acc8df-d4f9,11/30/20 18:01,12/26/20 3:16,level_event,"{duration=0, level=64, progress=37, play_count...",ios,12,0,"iPhone12,8",iOS 14.2,0.870634
2,684c9a99-aeb0,11/30/20 15:06,12/26/20 0:53,level_event,"{duration=39.537, level=7, progress=31, play_c...",ios,7,1,"iPhone11,8",iOS 14.3,0.183914
3,684c9a99-aeb0,11/30/20 15:06,12/26/20 0:53,level_event,"{duration=0, level=58, progress=31, play_count...",ios,7,1,"iPhone11,8",iOS 14.3,0.183914
4,be725f9b-33c5,11/30/20 3:20,11/30/20 3:21,interstitial_impression,{'revenue': '0.035'},ios,1,1,"iPhone12,1",iOS 14.0.1,0.036200


### Parsing event data
Getting each event type

In [4]:
df["eventType"].unique()

array(['level_event', 'interstitial_impression', 'banner_impression',
       'rewarded_impression'], dtype=object)

In [5]:
df["eventType"]= df["eventType"].astype(str)

Getting the data structure for each event type

In [6]:
for uniqevent in df["eventType"].unique():
    print(uniqevent, ":", df[df["eventType"]==uniqevent]["event_data"].iloc[100])

level_event : {duration=1.606702E+09, level=5, progress=42, play_count=42, status=1}
interstitial_impression : {'revenue': '0.03'}
banner_impression : {'revenue': '0.0003'}
rewarded_impression : {'revenue': '0.0275'}


Designing parsers for each data format

In [7]:
def extract_level_event_data(r):
    
    r = r.replace("=",":").replace('duration', '"duration"').replace("level", '"level"').replace("progress", '"progress"').replace("play_count", '"play_count"').replace("status", '"status"')
    r = json.loads(r)
    
    returndict = {
        "duration":0,
        "level":0,
        "progress":0,
        "play_count":0,
        "status":0,
    }
    
    returndict.update(r)
    
    return returndict["duration"], returndict["level"], returndict["progress"], returndict["play_count"], returndict["status"]

In [8]:
def extract_revenue_data(r):
    
    revenue = r.split(":")[1].split("'")[1]
    
    return revenue

Extracting event_data to seperate columns

In [9]:
df["duration"] = df["level"] = df["progress"] = df["play_count"] = df["status"] = df["revenue"] = 0

## extracting level data
df.loc[df["eventType"]=="level_event", ["duration", "level", "progress", "play_count", "status"]] =\
list(df['event_data'][df["eventType"]=="level_event"].apply(extract_level_event_data))

## extracting revenue data
df.loc[df["eventType"]!="level_event", "revenue"] =\
df['event_data'][df["eventType"]!="level_event"].apply(extract_revenue_data)

## object to float conversion
df["revenue"]= df["revenue"].astype(float)

Dropping json-like event_data

In [10]:
df.drop('event_data', 1, inplace=True)

In [11]:
df.head()

,userID_anonymized,arrival_time,event_time,eventType,platform,session_no,wifi_state,model,operating_system,totalrevenue,duration,level,progress,play_count,status,revenue
0,12acc8df-d4f9,11/30/20 18:01,12/26/20 3:16,level_event,ios,12,0,"iPhone12,8",iOS 14.2,0.870634,30.328,47,36,36,1,0.000
1,12acc8df-d4f9,11/30/20 18:01,12/26/20 3:16,level_event,ios,12,0,"iPhone12,8",iOS 14.2,0.870634,0.000,64,37,37,2,0.000
2,684c9a99-aeb0,11/30/20 15:06,12/26/20 0:53,level_event,ios,7,1,"iPhone11,8",iOS 14.3,0.183914,39.537,7,31,31,1,0.000
3,684c9a99-aeb0,11/30/20 15:06,12/26/20 0:53,level_event,ios,7,1,"iPhone11,8",iOS 14.3,0.183914,0.000,58,31,31,2,0.000
4,be725f9b-33c5,11/30/20 3:20,11/30/20 3:21,interstitial_impression,ios,1,1,"iPhone12,1",iOS 14.0.1,0.036200,0.000,0,0,0,0,0.035


### Extracting meaningful event time feature

In [12]:
df["arrival_time"] = pd.to_datetime(df["arrival_time"])
df["event_time"] = pd.to_datetime(df["event_time"])

### Leaving only first 3 day events

In [13]:
days = (df["event_time"]-df["arrival_time"]).astype('timedelta64[D]')
df = df[days<=3]

Extracting customer lifetimes in minute domain.

In [14]:
df["event_delay"] = (df["event_time"]-df["arrival_time"]).astype('timedelta64[m]')
df.drop(['arrival_time', 'event_time'], 1, inplace=True)

### Reordering the columns

In [15]:
# Dropping the 'platform' since they are all = "ios"
df = df[["userID_anonymized", "model", "operating_system", "wifi_state", "session_no", "event_delay", "eventType", "duration", "level", "progress", "play_count", "status", "revenue", "totalrevenue"]]

In [16]:
df.head()

,userID_anonymized,model,operating_system,wifi_state,session_no,event_delay,eventType,duration,level,progress,play_count,status,revenue,totalrevenue
4,be725f9b-33c5,"iPhone12,1",iOS 14.0.1,1,1,1.0,interstitial_impression,0.000000e+00,0,0,0,0,0.03500,0.036200
5,dd5ef583-258b,"iPhone12,1",iOS 14.2,0,13,195.0,banner_impression,0.000000e+00,0,0,0,0,0.00030,0.460578
6,be725f9b-33c5,"iPhone12,1",iOS 14.0.1,1,1,2.0,level_event,1.606707e+09,25,10,10,1,0.00000,0.036200
7,fb8331dd-f7f7,"iPhone11,8",iOS 14.2,1,3,45.0,banner_impression,0.000000e+00,0,0,0,0,0.00030,1.161082
8,64dea955-8f14,"iPhone11,6",iOS 14.2,1,1,3.0,banner_impression,0.000000e+00,0,0,0,0,0.00015,0.408266


# Descriptive Analysis on Preprocessed Time Series Data

In [17]:
df.describe()

,wifi_state,session_no,event_delay,duration,level,progress,play_count,status,revenue,totalrevenue
count,664469.000000,664469.000000,664469.000000,6.644690e+05,664469.000000,664469.000000,664469.000000,664469.000000,664469.000000,664469.000000
mean,0.770340,3.046338,852.146538,6.452515e+08,21.993533,15.471104,15.469196,0.402979,0.003523,0.724801
std,0.420615,3.318114,1340.488521,7.876748e+08,77.794183,35.676937,35.677131,0.490497,0.009190,0.629003
min,0.000000,1.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000200
25%,1.000000,1.000000,5.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.295700
50%,1.000000,2.000000,73.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000231,0.566566
75%,1.000000,4.000000,1286.000000,1.606758e+09,30.000000,15.000000,15.000000,1.000000,0.000600,0.975983
max,1.000000,67.000000,5759.000000,1.607125e+09,1010.000000,587.000000,587.000000,1.000000,0.317145,7.477914


In [18]:
df.describe(include=['object', 'bool'])

,userID_anonymized,model,operating_system,eventType
count,664469,664469,664469,664469
unique,8327,78,53,4
top,c3dcc33c-4c82,"iPhone12,1",iOS 14.2,banner_impression
freq,1449,107374,397663,305828


In [19]:
df["model"].value_counts()[:10]

iPhone12,1    107374
iPhone11,8    104894
iPhone12,8     44198
iPhone9,1      39704
iPhone8,1      27437
iPhone10,2     25892
iPhone10,1     25446
iPhone12,5     24813
iPhone11,6     15673
iPhone10,4     14567
Name: model, dtype: int64

In [20]:
df["operating_system"].value_counts()[:10]

iOS 14.2      397663
iOS 14.1       67732
iOS 14.0.1     37694
iOS 13.7       24836
iOS 14.0       24024
iOS 13.6.1     15127
iOS 12.4.9     13917
iOS 14.2.1     13537
iOS 13.5.1     13296
iOS 13.6       11859
Name: operating_system, dtype: int64

In [21]:
df["eventType"].value_counts()[:10]

banner_impression          305828
level_event                267767
interstitial_impression     86353
rewarded_impression          4521
Name: eventType, dtype: int64

Rewarded ads bring more revenue than interstitials, as expected.

In [22]:
for uniqevent in df["eventType"].unique():
    print("Revenues for", uniqevent, ">> mean:", np.round(df[df["eventType"]==uniqevent]["revenue"].mean(),3),\
         "|| std:", np.round(df[df["eventType"]==uniqevent]["revenue"].std(),3))

Revenues for interstitial_impression >> mean: 0.024 || std: 0.011
Revenues for banner_impression >> mean: 0.0 || std: 0.0
Revenues for level_event >> mean: 0.0 || std: 0.0
Revenues for rewarded_impression >> mean: 0.033 || std: 0.016


In [23]:
df.head()

,userID_anonymized,model,operating_system,wifi_state,session_no,event_delay,eventType,duration,level,progress,play_count,status,revenue,totalrevenue
4,be725f9b-33c5,"iPhone12,1",iOS 14.0.1,1,1,1.0,interstitial_impression,0.000000e+00,0,0,0,0,0.03500,0.036200
5,dd5ef583-258b,"iPhone12,1",iOS 14.2,0,13,195.0,banner_impression,0.000000e+00,0,0,0,0,0.00030,0.460578
6,be725f9b-33c5,"iPhone12,1",iOS 14.0.1,1,1,2.0,level_event,1.606707e+09,25,10,10,1,0.00000,0.036200
7,fb8331dd-f7f7,"iPhone11,8",iOS 14.2,1,3,45.0,banner_impression,0.000000e+00,0,0,0,0,0.00030,1.161082
8,64dea955-8f14,"iPhone11,6",iOS 14.2,1,1,3.0,banner_impression,0.000000e+00,0,0,0,0,0.00015,0.408266


## Saving pre-processed time-series data

In [24]:
df.to_csv('preprocessed_ts.csv', index=False) 